In [ ]:
import os
import pandas as pd
import plotly.express as px

DATASET_PATH = f'/home/andrew/smoking/data/thrasher'
RECORDING_IDS = os.listdir(DATASET_PATH)
RECORDING_PATH = f'{DATASET_PATH}/{RECORDING_IDS[0]}/raw/{RECORDING_IDS[0]}.0.csv'
df = pd.read_csv(RECORDING_PATH,skiprows=1)
df = df.reset_index()
fs = 20
df.timestamp = (df.timestamp - df.timestamp[0])*1e-9
df['index'] = df['index']/(fs*60) # index in minutes

In [ ]:
import torch
from torch import nn
X = torch.from_numpy(df[['acc_x','acc_y','acc_z']].to_numpy())
x = X[:,0].unsqueeze(1)
y = X[:,1].unsqueeze(1)
z = X[:,2].unsqueeze(1)
xs = [x[:-99]]
ys = [y[:-99]]
zs = [z[:-99]]
for i in range(1,99):
    xs.append(x[i:i-99])
    ys.append(y[i:i-99])
    zs.append(z[i:i-99])
xs.append(x[99:])
ys.append(y[99:])
zs.append(z[99:])
xs = torch.cat(xs,axis=1).float()
ys = torch.cat(ys,axis=1).float()
zs = torch.cat(zs,axis=1).float()
X = torch.cat([xs,ys,zs],axis=1)

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

class Casey1p0():
    def __init__(self):
        self.range = pd.read_csv('/home/andrew/smoking/data/casey_network/range',header=None).to_numpy()
        self.fc1 = pd.read_csv('/home/andrew/smoking/data/casey_network/input',header=None).to_numpy()
        self.fc2 = pd.read_csv('/home/andrew/smoking/data/casey_network/hidden',header=None).to_numpy()
    def __oldMinMaxNorm(self,x):
        return ((x-x.min())/(x.max()-x.min())).tolist()
    def __minMaxNorm(self,x):
        return ((2*((x-self.range[:,0])/(self.range[:,1]-self.range[:,0])))-1).tolist()
    def __tanSigmoid(self,x):
        output = []
        for xi in x:
            output.append((2/(1+np.exp(-2*xi)))-1)
        return output
    def __logSigmoid(self,x):
        return (1/(1+np.exp(-1*x)))
    def __forwardSingleBatch(self,x):
        x = [1] + self.__oldMinMaxNorm(x)
        x = [1] + self.__tanSigmoid(self.fc1 @ x)
        x = self.fc2 @ x
        x = self.__logSigmoid(x[0])
        return x
    def __call__(self,x):
        output = []
        for xi in tqdm(x):
            output.append(self.__forwardSingleBatch(xi))
        return output + [0]*99
model = Casey1p0()

In [57]:
y_pred = model(X)
df['y_pred'] = y_pred

In [58]:
fig = px.line(df.iloc[::2],y=['rawlabel','y_pred'])
fig.show(renderer='browser')

100%|██████████| 109997/109997 [00:05<00:00, 21987.64it/s]
